# Question A.1
## A.1.1

In [1]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
en_corpus = sc.textFile("/home/emma/Documents/LDSA/Assignment_2/Corpus/europarl-v7.sv-en.en")
en_corpus.cache()
lines_en = en_corpus.count()
print("Lines in EN corpus: ",lines_en)

Lines in EN corpus:  1862234


## A.1.2

In [2]:
sv_corpus = sc.textFile("/home/emma/Documents/LDSA/Assignment_2/Corpus/europarl-v7.sv-en.sv")
sv_corpus.cache()
lines_sv = sv_corpus.count()
print("Lines in SV corpus: ",lines_sv)

Lines in SV corpus:  1862234


## A.1.3

In [3]:
print("Equal number of lines: ")
lines_en == lines_sv

Equal number of lines: 


True

## A.1.4

In [4]:
print("Number of partitions, EN: ",en_corpus.getNumPartitions())
print("Number of partitions, SV: ",sv_corpus.getNumPartitions())

Number of partitions, EN:  9
Number of partitions, SV:  9


# Question A.2
## A.2.1

In [5]:
def preprocess(line):
    return line.lower()

In [6]:
en_corpus_low = en_corpus.filter(lambda x: not x.startswith("<")).map(preprocess)
sv_corpus_low = sv_corpus.filter(lambda x: not x.startswith("<")).map(preprocess)

## A.2.2

In [7]:
for line in en_corpus_low.take(100):
    print(line)

resumption of the session
i declare resumed the session of the european parliament adjourned on friday 17 december 1999, and i would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.
although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.
you have requested a debate on this subject in the course of the next few days, during this part-session.
in the meantime, i should like to observe a minute' s silence, as a number of members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the european union.
please rise, then, for this minute' s silence.
(the house rose and observed a minute' s silence)
madam president, on a point of order.
you will be aware from the press and television that there have been a number of bomb explosions and

## A.2.3
Number of lines after preprocessing:

In [8]:
print("Lines in EN corpus: ",en_corpus_low.count())
print("Lines in SV corpus: ", sv_corpus_low.count())

Lines in EN corpus:  1862234
Lines in SV corpus:  1862234


# Question A.3
## A.3.1
Split into words, map into (word, 1) pairs, reduce and sort by counts.

In [9]:
EN_counts = en_corpus_low.flatMap(lambda x: x.split(" ")).map(lambda y: (y, 1)) \
                        .reduceByKey(lambda a,b: a+b).sortBy(lambda x: -x[1])
SV_counts = sv_corpus_low.flatMap(lambda x: x.split(" ")).map(lambda y: (y, 1)) \
                        .reduceByKey(lambda a,b: a+b).sortBy(lambda x: -x[1])

## A.3.2

In [10]:
print("Top 10 words in EN corpus:")
for line in EN_counts.take(10):
    print(line)

print("Top 10 words in SV corpus:")
for line in SV_counts.take(10):
    print(line)

Top 10 words in EN corpus:
('the', 3498375)
('of', 1659758)
('to', 1539760)
('and', 1288401)
('in', 1085993)
('that', 797516)
('a', 773522)
('is', 758050)
('for', 534242)
('we', 522849)
Top 10 words in SV corpus:
('att', 1706293)
('och', 1344830)
('i', 1050774)
('det', 924866)
('som', 913276)
('för', 908680)
('av', 738068)
('är', 694381)
('en', 620310)
('vi', 539797)


# Question A.4
## A.4.1
Index the lines and combine lines with the same index.

In [11]:
en_corpus_nr = en_corpus_low.zipWithIndex().map(lambda x: (x[1], x[0]))
sv_corpus_nr = sv_corpus_low.zipWithIndex().map(lambda x: (x[1], x[0]))

In [12]:
combined = en_corpus_nr.join(sv_corpus_nr)
for line in combined.take(5):
    print(line)

(0, ('resumption of the session', 'återupptagande av sessionen'))
(1835010, ('i hope that he will take a little time to address this issue before we close.', 'jag hoppas att han tar sig litet tid för att gå in på det.'))
(1310724, ('they must be able to make an informed choice on the basis of clear information.', 'de måste få möjlighet att göra ett upplyst val med utgångspunkt från tydlig information.'))
(786438, ('i believe that it would be desirable for sis ii to be up and running as soon as possible and i feel that the commission should notify parliament of any delays that occur and the reasons behind those delays.', 'det är önskvärt att sis ii installeras och tas i drift så snart som möjligt, och jag anser att kommissionen bör meddela parlamentet om det uppstår eventuella förseningar och skälen till dem.'))
(262152, ('the distribution of automobiles has always had special treatment, a sectoral treatment, and, if pushed, i would say that this system may have made sense before, when 

Split sentences into words and remove sentences without a match, with more than 10 words or where the number of words in both languages don't match.

In [13]:
comb_words = combined.map(lambda x: (x[0], x[1][0].split(" "), x[1][1].split(" "))) \
                        .filter(lambda x: len(x[1]) > 0 and len(x[2]) > 0) \
                        .filter(lambda x: len(x[1]) <= 10 and len(x[2]) <= 10) \
                        .filter(lambda x: len(x[1]) == len(x[2]))

In [14]:
print(comb_words.count())
for line in comb_words.take(5):
    print(line)

85499
(1048698, ['in', 'reality,', 'very', 'few', 'meet', 'strangers', 'off', 'the', 'internet.'], ['mycket', 'få', 'träffar', 'främmande', 'kontakter', 'från', 'nätet', 'i', 'verkligheten.'])
(786564, ['corrections', 'to', 'votes', 'and', 'voting', 'intentions:', 'see', 'minutes'], ['rättelser', 'till', 'avgivna', 'röster', 'och', 'röstavsikter:', 'se', 'protokollet'])
(1835298, ['a', 'mandate', 'to', 'search', 'for', 'peace', 'in', 'our', 'country.'], ['ett', 'mandat', 'för', 'att', 'försöka', 'åstadkomma', 'fred', 'i', 'landet.'])
(1048986, ['this', 'will', 'improve', 'support', 'within', 'the', 'industry.'], ['det', 'kommer', 'att', 'öka', 'stödet', 'inom', 'industrin.'])
(1311156, ['we', 'must', 'act.'], ['vi', 'måste', 'agera.'])


Zip the sentences together and reduce to count the most common word pairs.

In [26]:
word_pairs = comb_words.flatMap(lambda x: zip(x[1], x[2])).map(lambda x: (x, 1)) \
                        .reduceByKey(lambda a,b: a+b).sortBy(lambda x: -x[1])

Most common word pairs:

In [30]:
for line in word_pairs.take(20):
    print(line)

(('is', 'är'), 11221)
(('we', 'vi'), 6594)
(('i', 'jag'), 6123)
(('and', 'och'), 3926)
(('this', 'detta'), 3782)
(('a', 'en'), 3496)
(('it', 'det'), 3416)
(('not', 'inte'), 3097)
(('that', 'det'), 3042)
(('closed.', 'avslutad.'), 2964)
(('(applause)', '(applåder)'), 2548)
(('in', 'i'), 2525)
(('have', 'har'), 2331)
(('a', 'ett'), 2246)
(('', '.'), 2223)
(('will', 'att'), 2218)
(('.', '.'), 2159)
(('are', 'är'), 2062)
(('the', 'omröstningen'), 2009)
(('vote', 'kommer'), 1955)


## A.4.2
Of the top 20 translations, most are correct. There are some errors with punctuation showing up, which could be filtrated out for slightly better results.